In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
# Load the saved CSV
df = pd.read_csv('../data/crypto_exchange_data.csv')
df

/var/folders/cz/_b8vfkr5561_03c0zfpnzbgc0000gn/T/ipykernel_52848/1301678430.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/crypto_exchange_data.csv')


,text,reddit_score,date,source,exchange,platform,rating,cleaned_text,sentiment,sentiment_score,...,security,coin_listings,performance,entities,keywords,is_duplicate,id,type,parent_id,word_count
0,It's a very reasonable return. She loaned them...,1.0,2022-08-30,r/CryptoCurrency,crypto.com,web,NaN,reasonable return loaned crypto dollar got bac...,negative,-0.6249,...,0.0000,-0.6249,0.0000,[],"[""bitcoin"", ""said"", ""cryptocom"", ""got"", ""need""...",False,371,comment,t3_x14ut4,24
1,"Yea I noticed Viacoin has a fee of 0,05 via wh...",19.0,2018-02-13,r/CryptoCurrency,binance,web,NaN,yea noticed viacoin fee via like c wtc fee req...,negative,-0.0772,...,0.0000,-0.0772,0.0000,[],"[""fee"", ""like""]",False,973,comment,t1_du4r3cq,17
2,Binance has a high risk of being locked out of...,1.0,2021-07-08,r/CryptoCurrency,binance,web,NaN,binance high risk locked major market yeah fir...,negative,-0.1027,...,0.0000,0.0000,0.0000,"['first', 'china eu uk']","[""market"", ""binance""]",False,2835,comment,t1_h4cddq6,13
3,Gemini all the way. Spent 3 months fucking wit...,60.0,2018-01-26,r/CryptoCurrency,kraken,web,NaN,gemini way spent month fucking krakens bullshi...,negative,-0.8916,...,-0.8916,0.0000,-0.8916,[],"[""transfer"", ""month"", ""way"", ""bank"", ""day"", ""k...",False,136,comment,t1_dt8c7it,20
4,Crypto exchanges should not allow shit coins i...,3.0,2022-04-13,r/CryptoCurrency,coinbase,web,NaN,crypto exchange allow shit coin imo decentrali...,negative,-0.6249,...,-0.0258,-0.1027,0.0000,[],"[""exchange"", ""sure"", ""youre"", ""scam"", ""company...",False,151,comment,t3_u2f90u,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33213,I’ve had a problem regarding my identity card ...,NaN,2023-06-24,trustpilot,bybit,web,4.0,ive problem regarding identity card used scamm...,neutral,0.0258,...,-0.2960,0.0000,0.0000,[],"[""problem"", ""customer"", ""support"", ""card"", ""us...",False,49317,trustpilot review,NaN,24
33214,Seems people come here to write a review only ...,NaN,2022-12-09,trustpilot,bybit,web,4.0,seems people come write review meet something ...,neutral,-0.0129,...,0.0000,0.0000,0.0000,[],"[""experience"", ""didnt"", ""reason"", ""thats"", ""us...",False,49332,trustpilot review,NaN,24
33215,Try to view my assets all I get are *****'s. W...,NaN,2023-02-12,trustpilot,bybit,web,1.0,try view asset get asset spot funding etc try ...,neutral,-0.0258,...,0.0000,-0.0258,0.0000,[],"[""asset"", ""used"", ""trade"", ""platform"", ""fund""]",False,49351,trustpilot review,NaN,17
33216,Bybit is wonderful to use and the overall func...,NaN,2023-01-28,trustpilot,bybit,web,1.0,bybit wonderful use overall functionality good...,neutral,-0.0000,...,0.3182,0.0000,0.0000,[],"[""order"", ""bybit"", ""scam"", ""trade"", ""good"", ""p...",False,49355,trustpilot review,NaN,39


In [4]:
# Define Custom Feature Lexicons
CRYPTO_FEATURES = {
    'Fees': [
        'fee', 'fees', 'trading fee', 'withdrawal fee', 'deposit fee',
        'low fee', 'high fee', 'expensive', 'cheap', 'commission',
        'hidden fee', 'transparent pricing', 'zero fee', 'cost', 'charge',
        'markup', 'processing fee', 'transaction cost', 'gas fee'
    ],
    'User Interface': [
        'app', 'website', 'ui', 'ux', 'interface', 'design', 'layout',
        'navigation', 'bug', 'glitch', 'slow', 'fast', 'responsive',
        'usability', 'mobile app', 'dashboard', 'update', 'dark mode',
        'intuitive', 'user-friendly', 'complicated', 'laggy', 'crashes',
        'loading', 'experience', 'accessibility'
    ],
    'Customer Service': [
        'support', 'help', 'customer', 'service', 'response', 'ticket',
        'chat', 'email', 'reply', 'unresponsive', 'delay', 'resolved',
        'complaint', 'agent', 'representative', 'live chat', 'call',
        'waiting time', 'inquiry', 'feedback', 'escalation', 'not helpful',
        'ignored', 'follow-up'
    ],
    'Security': [
        'secure', 'security', 'hack', 'breach', 'phishing', '2fa',
        'safety', 'account locked', 'withdrawal lock', 'verify',
        'verification', 'suspicious activity', 'identity theft', 'scam',
        'fraud', 'authentication', 'kyc', 'malware', 'cold wallet',
        'hot wallet', 'security token', 'ddos', 'data leak'
    ],
    'Coin Listings': [
        'listed', 'coin', 'token', 'listing', 'altcoin', 'available',
        'supported', 'delist', 'new coin', 'available pairs',
        'cryptocurrency', 'asset', 'stablecoin', 'pairing', 'market pair',
        'trading pair', 'not available', 'support for', 'removed',
        'launched', 'integrated'
    ],
    'Performance': [
        'crash', 'slow', 'lag', 'error', 'fail', 'stable', 'reliable',
        'outage', 'downtime', 'performance', 'uptime', 'maintenance',
        'server issue', 'connection lost', 'timeout', 'freeze', 'buggy',
        'high latency', 'stuck', 'reboot', 'real-time', 'speed'
    ]
}

text_column = 'cleaned_text'

# Count total number of records
num_records = len(df)

# Word count per row
df['word_count'] = df[text_column].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)

# Total words in corpus
total_words = df['word_count'].sum()

# Count unique words (types)
all_words = ' '.join(df[text_column].fillna('').astype(str)).split()
unique_words = set(all_words)
num_types = len(unique_words)

# Get categorical distributions
distributions = {}
for column in ['source', 'sentiment', 'platform', 'exchange']:
    if column in df.columns:
        distributions[f"{column}_distribution"] = df[column].value_counts().to_dict()

# Get feature score stats
feature_stats = {}
for feature in CRYPTO_FEATURES.keys():
    if feature in df.columns:
        feature_stats[feature] = {
            'mean': round(df[feature].mean(), 4),
            'median': round(df[feature].median(), 4),
            'std': round(df[feature].std(), 4),
            'min': round(df[feature].min(), 4),
            'max': round(df[feature].max(), 4)
        }

# Get text length stats
text_length_stats = {
    'mean_word_count': df['word_count'].mean(),
    'median_word_count': df['word_count'].median(),
    'min_word_count': df['word_count'].min(),
    'max_word_count': df['word_count'].max()
}

# Compile full statistics summary
statistics = {
    'num_records': num_records,
    'total_words': total_words,
    'num_unique_words': num_types,
    'lexical_diversity': num_types / total_words if total_words > 0 else 0,
    'distributions': distributions,
    'feature_stats': feature_stats,
    'text_length_stats': text_length_stats
}

In [5]:
# --- Display Output ---

# Overview
print("=== Dataset Overview ===")
pprint({
    'Total Records': statistics['num_records'],
    'Total Words': statistics['total_words'],
    'Unique Words': statistics['num_unique_words'],
    'Lexical Diversity': round(statistics['lexical_diversity'], 4)
})

# Distributions
print("\n=== Categorical Distributions ===")
for name, dist in statistics['distributions'].items():
    print(f"\n{name.replace('_', ' ').title()}:")
    for label, count in dist.items():
        print(f"  {label}: {count}")

# Text length stats
print("\n=== Text Length Statistics ===")
for stat, val in statistics['text_length_stats'].items():
    print(f"{stat.replace('_', ' ').title()}: {round(val, 2)}")

# Feature score stats (DataFrame)
if feature_stats:
    feature_df = pd.DataFrame(statistics['feature_stats']).T
    feature_df = feature_df[['mean', 'median', 'std', 'min', 'max']].round(3)
    print("\n=== Feature Sentiment Scores ===")
    display(feature_df)

# Optional: Plot sentiment distribution if available
if 'Sentiment_distribution' in statistics['distributions']:
    pd.Series(statistics['distributions']['Sentiment_distribution']).plot(kind='bar', title='Sentiment Distribution', ylabel='Count')
    plt.show()

=== Dataset Overview ===
{'Lexical Diversity': 0.0358,
 'Total Records': 33218,
 'Total Words': 1197819,
 'Unique Words': 42857}

=== Categorical Distributions ===

Source Distribution:
  r/CoinBase: 14123
  trustpilot: 8174
  r/binance: 2731
  r/CryptoCurrency: 2282
  r/BitcoinBeginners: 987
  r/kucoin: 889
  r/Crypto_com: 871
  r/Kraken: 773
  r/Bybit: 701
  app store: 554
  r/Ethereum: 349
  r/CryptoScams: 318
  r/OKX: 231
  play store: 129
  r/CryptoTechnology: 68
  r/CryptoMarkets: 38

Sentiment Distribution:
  negative: 14400
  positive: 14400
  neutral: 4418

Platform Distribution:
  web: 32535
  ios: 554
  android: 129

Exchange Distribution:
  coinbase: 15431
  binance: 5840
  kraken: 2855
  crypto.com: 2679
  bybit: 1885
  kucoin: 1629
  okx: 974
  binance, coinbase: 654
  coinbase, kraken: 377
  coinbase, crypto.com: 144
  binance, coinbase, kraken: 125
  binance, kraken: 122
  binance, kucoin: 104
  binance, crypto.com: 74
  coinbase, kucoin: 67
  binance, bybit: 37
  binan